In [15]:
import pandas as pd
import nltk
import spacy
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

In [16]:
import pandas as pd

# Cargar los datos de entrenamiento y prueba
train_path = "FinancES_train_kaggle.csv"  # Ruta del archivo de entrenamiento
test_path = "FinancES_test_kaggle.csv"    # Ruta del archivo de prueba

# Leer los archivos CSV
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

# Mostrar información de los datos cargados
print("Train Dataset:")
print(train_df.info())
print(train_df.head())

print("\nTest Dataset:")
print(test_df.info())
print(test_df.head())

Train Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      6359 non-null   int64 
 1   text    6359 non-null   object
 2   label   6359 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 149.2+ KB
None
   id                                               text  label
0   0  Renfe afronta mañana un nuevo día de paros par...      2
1   1       Presupuesto populista con cimientos frágiles      2
2   2  Biden no cree que la OPEP+ vaya a ayudar con l...      2
3   3  La deuda de las familias cae en 25.000 millone...      0
4   4  Bestinver: no hay "momento más inoportuno" par...      2

Test Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1621 non-null   int64 
 1   text    1621 non-null   

In [17]:
# Exploracion

# Identificar distribución de las clases
print("\nDistribución de las clases en el conjunto de entrenamiento:")
print(train_df["label"].value_counts())

# Verificar si hay valores nulos
print("\nValores nulos en el conjunto de entrenamiento:")
print(train_df.isnull().sum())

print("\nValores nulos en el conjunto de prueba:")
print(test_df.isnull().sum())



Distribución de las clases en el conjunto de entrenamiento:
label
2    2935
0    2818
1     606
Name: count, dtype: int64

Valores nulos en el conjunto de entrenamiento:
id       0
text     0
label    0
dtype: int64

Valores nulos en el conjunto de prueba:
id      0
text    0
dtype: int64


In [18]:
# Análisis de longitud de los textos
train_df["text_length"] = train_df["text"].apply(len)

print("\nEstadísticas de la longitud de los textos en el conjunto de entrenamiento:")
print(train_df["text_length"].describe())


Estadísticas de la longitud de los textos en el conjunto de entrenamiento:
count    6359.000000
mean       72.736751
std        20.955372
min         3.000000
25%        61.000000
50%        72.000000
75%        84.000000
max       211.000000
Name: text_length, dtype: float64


In [19]:
import re

# Normalización: convertir a minúsculas
train_df["text"] = train_df["text"].str.lower()
test_df["text"] = test_df["text"].str.lower()

print("\nTexto preprocesado:")
print(train_df["text"].head())



Texto preprocesado:
0    renfe afronta mañana un nuevo día de paros par...
1         presupuesto populista con cimientos frágiles
2    biden no cree que la opep+ vaya a ayudar con l...
3    la deuda de las familias cae en 25.000 millone...
4    bestinver: no hay "momento más inoportuno" par...
Name: text, dtype: object


In [20]:
import pandas as pd
import nltk
import spacy
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re

# Descargar recursos necesarios
nltk.download('punkt')
nltk.download('stopwords')

# Cargar los datos de entrenamiento y prueba
train_path = "FinancES_train_kaggle.csv"  # Ruta del archivo de entrenamiento
test_path = "FinancES_test_kaggle.csv"    # Ruta del archivo de prueba

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Pablo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
from nltk.stem import SnowballStemmer
from collections import Counter

# Contar palabras más frecuentes en el dataset
all_words = " ".join(train_df["text"]).split()
word_freq = Counter(all_words)

# Eliminar palabras que aparecen en más del 80% de los documentos
frequent_words = {word for word, freq in word_freq.items() if freq > 0.8 * len(train_df)}

# Inicializar el Stemmer para español
stemmer = SnowballStemmer("spanish")

# Lista de stopwords personalizada
custom_stopwords = set(stopwords.words('spanish'))

# Agregar palabras irrelevantes para titulares financieros
custom_stopwords.update({"día", "años", "mes", "nuevo", "euros"})

# Añadir palabras frecuentes a la lista de stopwords
custom_stopwords.update(frequent_words)

# Función de preprocesamiento con Stemming y stopwords mejoradas
def preprocess_text(text):
    
    # Tokenización
    tokens = word_tokenize(text)
    
    # Eliminar signos de puntuación y caracteres especiales, manteniendo números
    tokens = [re.sub(r"(?<!\d)[^\w\s](?!\d)", "", token) for token in tokens]
    
    # Eliminar stopwords personalizadas
    tokens = [word for word in tokens if word not in custom_stopwords]
    
    # Aplicar Stemming
    tokens = [stemmer.stem(word) for word in tokens]
    
    return " ".join(tokens)

# Aplicar preprocesamiento
train_df["text"] = train_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)


In [46]:
# Mostrar información de los datos cargados
print("Train Dataset:")
print(train_df.info())
print(train_df.head())

print("\nTest Dataset:")
print(test_df.info())
print(test_df.head())

Train Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6359 entries, 0 to 6358
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           6359 non-null   int64 
 1   text         6359 non-null   object
 2   label        6359 non-null   int64 
 3   text_length  6359 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 198.8+ KB
None
   id                                               text  label  text_length
0   0        renf afront mañ nuev dia par parcial maquin      2           71
1   1                    presupuest popul cimient fragil      2           44
2   2                 bid cre opep vay ayud preci petrol      2           69
3   3  deud famili caer 25000 millon 2015 marc nivel ...      0           91
4   4                    bestinv moment inoportun brexit      2           57

Test Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1621 entries, 0 to 1620
Data columns (total 3 col

In [55]:
# Vectorización TF-IDF
vectorizer = TfidfVectorizer(ngram_range=(1, 3), norm='l2')
X = vectorizer.fit_transform(train_df["text"])
y = train_df["label"]

# Separar conjunto de entrenamiento y prueba
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Aplicar SMOTE para balancear las clases en el conjunto de entrenamiento
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Entrenar modelo SVM con los datos balanceados
svm_model = SVC(kernel='linear', class_weight='balanced', random_state=42, C=10)
svm_model.fit(X_train_resampled, y_train_resampled)

# Evaluar el modelo
y_pred = svm_model.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print(classification_report(y_val, y_pred))

# Transformar los datos de test y generar predicciones
X_test = vectorizer.transform(test_df["text"])
test_predictions = svm_model.predict(X_test)

test_df["label"] = test_predictions
print(test_df.head())

Accuracy: 0.675314465408805
              precision    recall  f1-score   support

           0       0.64      0.74      0.69       546
           1       0.86      0.24      0.37       127
           2       0.70      0.71      0.70       599

    accuracy                           0.68      1272
   macro avg       0.73      0.56      0.59      1272
weighted avg       0.69      0.68      0.66      1272

   id                                               text  label
0   0         empres chin ped men burocraci invert españ      2
1   1  enriqu escal hav trabaj soft skills mund cambi...      0
2   2  banc march conf gestion empresarial nub microsoft      0
3   3         garamendi ruptur fusion noviazg lueg retom      0
4   4            gobiern vasc dar hech continu euskaltel      0


In [56]:
if True:
    # Crear archivo CSV con el formato requerido
    test_df[['id', 'label']].to_csv("submission.csv", index=False)

    print("Archivo 'submission.csv' creado correctamente.")

Archivo 'submission.csv' creado correctamente.
